In [1]:
import torch
import torchvision as tv

In [3]:
model = tv.models.vgg19()

In [6]:
test = model.children()

In [8]:
data = tv.datasets.CIFAR100('../data', download=True)

100.0%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
from torchvision.datasets import CIFAR10

# Load a pre-trained model, e.g., VGG19
model = models.vgg19(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Remove the last classification layer to use the remaining layers as a feature extractor
features = nn.Sequential(*list(model.children())[:-1])

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset, e.g., CIFAR10
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Function to extract features
def extract_features(data_loader, feature_extractor):
    features_list = []
    labels_list = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            features = feature_extractor(inputs)
            features_list.append(features.view(features.size(0), -1))
            labels_list.append(labels)
    return torch.cat(features_list), torch.cat(labels_list)

# Extract features
X_train, y_train = extract_features(train_loader, features)

# Create a TensorDataset and DataLoader for the features
feature_dataset = TensorDataset(X_train, y_train)
feature_loader = DataLoader(feature_dataset, batch_size=64, shuffle=True)

# Define a linear classifier
class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LinearClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

# Initialize the linear classifier
input_dim = X_train.size(1)
num_classes = 10  # For CIFAR10
linear_classifier = LinearClassifier(input_dim, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_classifier.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in feature_loader:
        optimizer.zero_grad()
        outputs = linear_classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Evaluate the classifier
linear_classifier.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in feature_loader:
        outputs = linear_classifier(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')